In [1]:
# https://ijassa.ipu.ru/index.php/ijassa/article/view/899/538


import random
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import numpy as np
import os
from pathlib import Path

from sem.generate_series import create_sde_process
from sem.sem.mixture_em_diff import NormalMixtureEM
from sem.sem.mixture_sem import MixtureSEM
from sem.sem.windows import find_window_size


class EMForecaster(nn.Module):
    def __init__(self, 
                 window_size: int,
                 n_components: int = 3,
                 hidden_dim: int = 64,
                 n_em_iters: int = 5):
        super().__init__()
        
        self.dwindow_size = window_size - 1
        self.window_size = window_size
        self.n_components = n_components
        negative_slope = 0.01

        self.em_layer = NormalMixtureEM(
            series_length=self.dwindow_size,
            n_components=n_components,
            n_sem_iters=n_em_iters
        )

        fusion_input_dim = n_components * self.dwindow_size + n_components * 5
        self.fusion = nn.Sequential(
            nn.Linear(fusion_input_dim, 2 * hidden_dim, bias=False),
            nn.LeakyReLU(negative_slope),
            nn.Linear(2 * hidden_dim, hidden_dim, bias=False),
            nn.LeakyReLU(negative_slope),
            nn.Linear(hidden_dim, 1, bias=False)
        )

        for module in self.modules():
            if isinstance(module, nn.Linear):
                nn.init.kaiming_uniform_(module.weight, negative_slope)
                if module.bias is not None:
                    nn.init.zeros_(module.bias)

    def forward(self, X_window):

        dX1 = X_window[..., 1:] - X_window[..., :-1]
        g_ik, p_k, a_k, b_k = self.em_layer(dX1)

        attention_logits = -b_k
        attention_weights = nn.functional.softmax(attention_logits, dim=1)

        fusion_input = torch.cat([
            (g_ik * dX1.unsqueeze(-1)).flatten(1),
            a_k,
            b_k,
            p_k,
            attention_weights * a_k,
            b_k * b_k * p_k
        ], dim=1)
        forecast = X_window[..., -1] + self.fusion(fusion_input).squeeze(-1)
        
        return forecast
    
    def forward_multistep(self, windows: torch.Tensor, n_steps: int) -> torch.Tensor:
        """
        Autoregressive multi-step forecasting for stateless models.
        
        Args:
            windows: [batch_size, window_size] - input windows
            n_steps: number of steps to forecast
            
        Returns:
            predictions: [batch_size, n_steps]
        """
        forecasts = []
        for _ in range(n_steps):
            forecast = self.forward(windows)
            forecasts.append(forecast)

            windows = torch.cat([
                windows[:, 1:],
                forecast.unsqueeze(-1)
            ], dim=1)
        results = torch.stack(forecasts, dim=1)
        return results
    
    @torch.no_grad()
    def generate_forecast(self, idx: int, 
                         time_series: torch.Tensor,
                         n_test_steps: int,
                         window_size: int) -> torch.Tensor:
        """
        Generate forecast starting at given index.
        
        Args:
            idx: starting index (negative values count from end)
            time_series: full time series data
            n_test_steps: number of steps to forecast
            window_size: size of input window
            train_series: alias for time_series (for backward compatibility)
            
        Returns:
            predictions: forecast values
        """
        if idx < 0:
            idx = time_series.shape[-1] + idx
        idx += 1

        start_idx = idx - window_size
        ts = time_series[..., start_idx: idx]
        assert ts.shape[-1] == window_size, 'not enough data'

        if not isinstance(ts, torch.Tensor):
            ts = torch.tensor(ts, dtype=torch.float32)
        if ts.dim() == 1:
            ts = ts.unsqueeze(0)

        device = next(self.parameters()).device
        window = ts.to(device)

        preds = self.forward_multistep(window, n_test_steps)
        if ts.shape[0] == 1:
            preds = preds.squeeze(0)
            
        return preds


class SEMForecaster(nn.Module):
    def __init__(self, 
                 window_size: int,
                 sem_method,
                 n_components: int = 3,
                 hidden_dim: int = 64):
        super().__init__()
        
        self.dwindow_size = window_size - 1
        self.window_size = window_size
        self.n_components = n_components
        self.sem_method = sem_method
        negative_slope = 0.01

        fusion_input_dim = n_components * self.dwindow_size + n_components * 5
        self.fusion = nn.Sequential(
            nn.Linear(fusion_input_dim, 2 * hidden_dim, bias=False),
            nn.LeakyReLU(negative_slope),
            nn.Linear(2 * hidden_dim, hidden_dim, bias=False),
            nn.LeakyReLU(negative_slope),
            nn.Linear(hidden_dim, 1, bias=False)
        )

        for module in self.modules():
            if isinstance(module, nn.Linear):
                nn.init.kaiming_uniform_(module.weight, negative_slope)
                if module.bias is not None:
                    nn.init.zeros_(module.bias)

    def forward(self, X_window, g_ik, p_k, a_k, b_k):
        dX1 = X_window[..., 1:] - X_window[..., :-1]

        attention_logits = -b_k
        attention_weights = nn.functional.softmax(attention_logits, dim=1)

        fusion_input = torch.cat([
            (g_ik * dX1.unsqueeze(-1)).flatten(1),
            a_k,
            b_k,
            p_k,
            attention_weights * a_k,
            b_k * b_k * p_k
        ], dim=1)
        forecast = X_window[..., -1] + self.fusion(fusion_input).squeeze(-1)
        
        return forecast

    def forward_multistep(self, windows: torch.Tensor, n_steps: int) -> torch.Tensor:
        """
        Autoregressive multi-step forecasting for stateless models.
        
        Args:
            windows: [batch_size, window_size] - input windows
            n_steps: number of steps to forecast
            
        Returns:
            predictions: [batch_size, n_steps]
        """
        forecasts = []
        for _ in range(n_steps):
            dwindows = windows[:, 1:] - windows[:, :-1]
            g_ik, w, mu, sigma = self.sem_method(dwindows)
            forecast = self.forward(windows, g_ik, w, mu, sigma)
            forecasts.append(forecast)

            windows = torch.cat([
                windows[:, 1:],
                forecast.unsqueeze(-1)
            ], dim=1)
        results = torch.stack(forecasts, dim=1)
        return results
    
    @torch.no_grad()
    def generate_forecast(self, idx: int, 
                         time_series: torch.Tensor,
                         n_test_steps: int,
                         window_size: int) -> torch.Tensor:
        """
        Generate forecast starting at given index.

        Args:
            idx: starting index (negative values count from end)
            time_series: full time series data
            n_test_steps: number of steps to forecast
            window_size: size of input window
            train_series: alias for time_series (for backward compatibility)

        Returns:
            predictions: forecast values
        """
        if idx < 0:
            idx = time_series.shape[-1] + idx
        idx += 1

        start_idx = idx - window_size
        ts = time_series[..., start_idx: idx]
        assert ts.shape[-1] == window_size, 'not enough data'

        if not isinstance(ts, torch.Tensor):
            ts = torch.tensor(ts, dtype=torch.float32)
        if ts.dim() == 1:
            ts = ts.unsqueeze(0)

        device = next(self.parameters()).device
        window = ts.to(device)

        preds = self.forward_multistep(window, n_test_steps)
        if ts.shape[0] == 1:
            preds = preds.squeeze(0)
            
        return preds

In [2]:
from time import time
import json

N_COMPONENTS = 3
TOL = 1e-4
TOL_TEST = 1e-4

class TimeSeriesDataset(Dataset):
    def __init__(self, time_series: np.ndarray, window_size, n_train_steps=1):
        super().__init__()
        self.ts = time_series
        self.n_train_steps = n_train_steps
        self.window_length = window_size

    def __len__(self):
        return len(self.ts) - self.window_length - self.n_train_steps

    def __getitem__(self, idx):
        idx_last = idx + self.window_length
        return self.ts[idx:idx_last], self.ts[idx_last: idx_last + self.n_train_steps]

class StaticMixtureTimeSeriesDataset(Dataset):
    def __init__(self, time_series: np.ndarray, window_size: int, comp_distr, n_train_steps=1):
        super().__init__()
        self.ts = time_series
        self.n_train_steps = n_train_steps
        self.window_length = window_size
        dwindow_size = window_size - 1
        sem = MixtureSEM(time_series[1:] - time_series[:-1], None, dwindow_size, n_components=N_COMPONENTS, comp_distr=comp_distr, tol=TOL)
        start = time()
        self.g, self.w, self.mu, self.sigma = sem.find_params()
        print(comp_distr, time() - start)

    def __len__(self):
        return self.g.shape[0] - 1

    def __getitem__(self, idx):
        idx_last = idx + self.window_length
        target = self.ts[idx_last: idx_last + self.n_train_steps]
        window = self.ts[idx:idx_last]

        return window, target, self.g[idx], self.w[idx], self.mu[idx], self.sigma[idx]


class MAPELoss(nn.Module):
    def __init__(self, eps=1e-8):
        super().__init__()
        self.eps = eps
    
    def forward(self, predictions, targets):
        mape = torch.abs(predictions - targets) / (torch.abs(targets) + self.eps)
        return mape.mean()


class WeightedLoss(nn.Module):
    def __init__(self, mape=0.1, mse=0.9, eps=1e-8):
        super().__init__()
        self.mape = MAPELoss(eps)
        self.mse = nn.MSELoss()
        self.mape_weight = mape
        self.mse_weight = mse

    def forward(self, predictions, targets):
        return self.mape_weight * self.mape(predictions, targets) + self.mse_weight * self.mse(predictions, targets)


class Trainer:
    def __init__(self, setup, time_series, window_size, len_train, log_path,
                 log_freq_batch=50, n_tests=100, n_test_steps=50, n_train_steps=1, eps=1e-8, device='cuda'):
        self.train_series = time_series[:len_train]
        self.eval_series = time_series[len_train:]
        self.dataloader = setup['dataloader']

        self.window_size = window_size
        self.n_epochs = 50
        self.n_tests = n_tests
        self.n_test_steps = n_test_steps
        self.model = setup['model'].to(device)
        self.optimizer = torch.optim.AdamW(self.model.parameters(), 1e-3)
        self.criterion = WeightedLoss()
        self.scheduler = torch.optim.lr_scheduler.LinearLR(
            self.optimizer, 1.0, 1e-6, self.n_epochs
        )

        self.loss_threshold = 0.01
        self.eps = eps
        self.device = device
        self.log_freq_batch = log_freq_batch
        self.log_path = log_path
        
        self.n_train_steps = n_train_steps

    @torch.no_grad()
    def validate(self):
        preds = self.model.generate_forecast(
            idx=-1,
            time_series=self.train_series,
            n_test_steps=self.n_test_steps,
            window_size=self.window_size
        ).cpu().numpy()
        target = self.eval_series[:self.n_test_steps]
        return preds, target
    
    @torch.no_grad()
    def evaluate_multistep(self, test_ids):
        preds = np.empty((self.n_tests + 1, self.n_test_steps))
        targets = np.empty((self.n_tests + 1, self.n_test_steps))
        for test in range(self.n_tests):
            idx = test_ids[test]
            target_idx = idx + 1
            pred = self.model.generate_forecast(
                idx=idx,
                time_series=self.train_series,
                n_test_steps=self.n_test_steps,
                window_size=self.window_size
            ).cpu()
            target = self.train_series[target_idx:target_idx + self.n_test_steps]
            preds[test, :] = pred.numpy()
            targets[test, :] = target
        return preds, targets

    def run(self, test_ids):
        epoch_train_time = 0
        epoch_val_time = 0
        start_full = time()
        for epoch in range(1, self.n_epochs + 1):
            self.model.train()
            start_train = time()
            avg_threshold = self.train_epoch(epoch)
            epoch_train_time += time() - start_train
            self.model.eval()

            start_test = time()
            preds, targets = self.evaluate_multistep(test_ids)
            pred, target = self.validate()
            epoch_val_time += time() - start_test
            preds[-1, :] = pred
            targets[-1, :] = target
            tests_results = np.stack([preds, targets], axis=-1)  # shape: [n_tests+1, n_steps, 2]
            
            np.save(self.log_path / f'epoch_{epoch}', tests_results)
            
            if avg_threshold < self.loss_threshold:
                break
        
        time_full = time() - start_full
        mean_train_time = epoch_train_time / epoch
        mean_test_time = epoch_val_time / epoch / (self.n_test_steps + 1) / (self.n_tests)

        json.dump(
                {
                    'full_time': float(time_full),
                    'epoch_train_time': float(mean_train_time),
                    'epoch_test_time_one_step': float(mean_test_time),
                    'last_loss': float(avg_threshold)
                },
                open(self.log_path / 'times.json', 'w'),
                indent=2
            )
        print()
        print("Training completed!")
        return self.model
    
    def train_epoch(self, epoch_idx=0):
        print("Epoch:", epoch_idx)
        total_loss = 0.0

        for batch_idx, batch in enumerate(self.dataloader):
            target = batch.pop(1).to(self.device).float()
            windows = batch.pop(0).to(self.device).float()
            if self.n_train_steps == 1:
                target = target.squeeze(-1)

            self.optimizer.zero_grad()

            if self.n_train_steps == 1:
                results = self.model.forward(windows, *batch)
                if isinstance(results, tuple):
                    results = results[0]
            else:
                results = self.model.forward_multistep(windows, self.n_train_steps)

            loss = self.criterion(results, target)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(self.model.parameters(), max_norm=0.5)
            self.optimizer.step()

            total_loss += loss.item()

            if batch_idx % self.log_freq_batch == 0:
                basic_metrics = self.compute_basic_metrics(results, target)
                print("Batch IDX:", batch_idx)
                print(basic_metrics)
                print()
            if torch.isnan(loss):
                print(f"WARNING: NaN loss at batch {batch_idx}")
                break
        self.scheduler.step()

        avg_loss = total_loss / len(self.dataloader)
        print(f"Epoch {epoch_idx} - Average Loss: {avg_loss:.6f}")
        return avg_loss

    @torch.no_grad()
    def compute_basic_metrics(self, forecast, y_true):
        mae = torch.abs(forecast - y_true).mean().item()
        mse = torch.mean((forecast - y_true) ** 2).item()
        mape = torch.abs((forecast - y_true) / (y_true.abs() + self.eps)).mean().item() * 100
        return {'MAE': mae, 'MSE': mse, 'MAPE': mape}


In [3]:
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

def get_setups(train_series, window_size, batch_size):
    dwindow_size = window_size - 1
    sem_methods = {
        'normal': MixtureSEM(None, None, dwindow_size, n_components=N_COMPONENTS, comp_distr='normal', tol=TOL_TEST)._fit_batch,
        'student': MixtureSEM(None, None, dwindow_size, n_components=N_COMPONENTS, comp_distr='student', tol=TOL_TEST)._fit_batch,
        'laplace': MixtureSEM(None, None, dwindow_size, n_components=N_COMPONENTS, comp_distr='laplace', tol=TOL_TEST)._fit_batch
    }

    sem_dataloaders = {
        distr: DataLoader(StaticMixtureTimeSeriesDataset(train_series,
                                                         window_size,
                                                         distr),
                          batch_size,
                          shuffle=True)
        for distr in sem_methods
    }
    sem_dataloaders['normal_diff'] = DataLoader(TimeSeriesDataset(train_series, window_size), batch_size, shuffle=True)

    models = [SEMForecaster(window_size, method, N_COMPONENTS, 64) for method in sem_methods.values()]
    models.append(EMForecaster(window_size, N_COMPONENTS, 64, 5))

    setups = []
    for distr, model in zip(sem_dataloaders.keys(), models):
        setups.append({
            'name': (f'SEM_{distr}' if 'diff' not in distr else f'EM_{distr}'),
            'model': model,
            'dataloader': sem_dataloaders[distr]
        })

    return setups

def create_experiment_folder(base_path="experiments", model_name=None, series_idx=None):
    """
    Create folder structure: experiments/<model_name>/<series_idx>/
    """
    if model_name is None:
        model_name = "default"
    if series_idx is None:
        series_idx = "0"
    
    exp_path = Path(base_path) / model_name / str(series_idx)
    exp_path.mkdir(parents=True, exist_ok=True)
    
    return exp_path

def generate_test_ids(window_size, train_series, n_tests, n_test_steps):
    res = []
    for _ in range(n_tests):
        res.append(random.randint(window_size, len(train_series) - 1 - n_test_steps))
    return res

In [4]:
n_tests = 10
series_length = 3200
train_length = 3000
alpha = 0.75
N_init = 5
N_add = 5
n_epoch_tests = 15
n_test_steps = 200
batch_size = 64

for test in range(n_tests):
    print('_' * 50)
    print(test)
    print('_' * 50)
    set_seed(10 * test)
    series = create_sde_process(series_length)['X']
    train_series = series[:train_length]
    *_, window_size = find_window_size(train_series[1:] - train_series[:-1], alpha, N_init, N_add)
    window_size += 1
    setups = get_setups(train_series, window_size, batch_size)
    test_ids = generate_test_ids(window_size, train_series, n_epoch_tests, n_test_steps)
    for setup in setups:
        set_seed(10 * test)
        print('_' * 50)
        print(setup['name'])
        print('_' * 50)

        log_path = create_experiment_folder(model_name=setup['name'], series_idx=test)
        if len(os.listdir(log_path)):
            print(log_path)
            continue
        trainer = Trainer(setup, series, window_size, train_length, log_path,
                          log_freq_batch=70, n_tests=n_epoch_tests, n_test_steps=n_test_steps, device='cuda')
        model = trainer.run(test_ids)

__________________________________________________
0
__________________________________________________
N = 5; Max ACF(1): 1.0
N = 10; Max ACF(1): 0.9714626669883728
N = 15; Max ACF(1): 0.8973427414894104
N = 20; Max ACF(1): 0.8615346550941467
N = 25; Max ACF(1): 0.8269753456115723
N = 30; Max ACF(1): 0.8037227988243103
N = 35; Max ACF(1): 0.796712338924408
N = 40; Max ACF(1): 0.7947536110877991
N = 45; Max ACF(1): 0.7923765778541565
N = 50; Max ACF(1): 0.7715969085693359
N = 55; Max ACF(1): 0.7685389518737793
N = 60; Max ACF(1): 0.7699713706970215
N = 65; Max ACF(1): 0.7635490894317627
N = 70; Max ACF(1): 0.7643201947212219
N = 75; Max ACF(1): 0.7623101472854614
N = 80; Max ACF(1): 0.7507982850074768
N = 85; Max ACF(1): 0.7455227375030518
Found window length: 85
normal 0.020441293716430664
student 0.1472158432006836
laplace 0.0594632625579834
__________________________________________________
SEM_normal
__________________________________________________
Epoch: 1
Batch IDX: 0
{'MAE': 1